In [4]:
import pandas as pd

ene_df = pd.read_csv("/home/deecodes/internship/AfricaEnergy/africa_energy_energy_data.csv")
social_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/energy/Database_Energy_Access_Energy_-Population-with-access-to-clean-cooking-fuels-(%-of-population).xlsx")

social_df.head(10)

,Country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Algeria,96.4,97.00,97.4,97.8,98.1,98.4,98.6,98.8,99.0,...,99.40,99.50,99.6,99.6,99.7,99.7,99.7,99.7,99.7,99.7
1,Angola,40.9,41.35,41.5,42.1,41.8,42.7,43.1,43.1,43.7,...,45.95,46.60,47.3,47.8,48.3,48.6,49.0,49.7,49.8,50.0
2,Benin,0.6,0.80,1.1,1.5,1.9,2.3,2.8,3.3,3.8,...,5.00,5.10,5.3,5.2,5.2,5.3,5.4,5.7,5.8,6.0
3,Botswana,44.2,46.20,47.9,49.3,51.0,52.1,53.4,54.8,56.3,...,60.50,61.40,61.8,62.5,63.0,63.7,64.4,64.8,65.4,66.0
4,Burkina Faso,2.7,2.90,3.1,3.4,3.6,3.9,4.3,4.6,5.0,...,7.50,8.20,8.9,10.0,10.8,11.9,13.1,14.5,15.8,17.2
5,Burundi,0.3,0.30,0.3,0.3,0.3,0.3,0.3,0.2,0.2,...,0.20,0.20,0.2,0.2,0.1,0.1,0.1,0.1,0.1,0.1
6,Cameroon,9.8,10.80,11.7,12.8,13.8,14.8,15.7,16.6,17.4,...,21.00,21.90,22.8,23.6,24.3,25.3,26.4,27.4,28.6,29.4
7,Cape Verde,62.8,63.00,63.4,63.7,64.3,64.6,65.6,66.5,67.4,...,73.00,74.35,75.5,76.7,77.9,79.0,80.2,81.1,82.0,83.0
8,Central African Republic,0.6,0.50,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.60,0.60,0.6,0.6,0.7,0.7,0.8,0.9,1.0,1.0
9,Chad,3.0,2.80,2.6,2.5,2.3,2.3,2.2,2.3,2.4,...,3.40,3.80,4.2,4.7,5.2,6.0,6.7,7.5,8.4,9.5


In [22]:
def fill_subsector_data_known(main, sub_data, subsector_name):
    """
    Fill missing values in main dataframe from sub_data for a specific subsector.
    
    Args:
        main: Main dataframe to update
        sub_data: Dataframe with additional data to merge
        subsector_name: Name of the subsector to filter on
    
    Returns:
        Updated main dataframe
    """
    try:
        # Rename columns (create copy to avoid modifying original)
        sub_data = sub_data.rename(columns={"Country": "country"})
        
        # Create mask for the subsector
        mask = main["sub_sub_sector"].str.lower() == subsector_name.lower()
        
        # Check if any rows match
        if not mask.any():
            print(f"Warning: No rows found for subsector '{subsector_name}'")
            return main
        
        # Create a mapping dictionary from sub_data
        country_map = sub_data.set_index("country")
        
        # Fill missing values for each year column
        for col in sub_data.columns:
            if col in main.columns and col != "country":
                # Fill NaN values in main with values from sub_data
                main.loc[mask, col] = main.loc[mask, col].fillna(
                    main.loc[mask, "country"].map(country_map[col])
                )
        
        print(f"Successfully filled data for subsector '{subsector_name}' ({mask.sum()} rows)")
        return main
        
    except Exception as e:
        print(f"Error in fill_subsector_data_known: {e}")
        return main

In [6]:
def replace_unknown(df, metric, unit, sub_sector, sub_sub_sector):
    try:
        # Replace 'Unknown' or 'unknown' in each column
        df["metric"] = df["metric"].replace(["Unknown", "unknown", "", "Installed Capacity in Fossil Fuels"], metric)
        df["unit"] = df["unit"].replace(["Unknown", "unknown", "", "MW"]).fillna(unit)
        df["sub_sector"] = df["sub_sector"].replace(["Unknown", "unknown", "", "Technical"], sub_sector)
        df["sub_sub_sector"] = df["sub_sub_sector"].replace(["Unknown", "unknown", "", "Electricity, Installed Capacity in Fossil Fuels"], sub_sub_sector)
        return df
    except Exception as e:
        print(f"Error in replace_unknown: {e}")
        return df

In [7]:
def fill_subsector_data_unknown(main, metric, unit, sub_sector, sub_sub_sector, sub_data, subsector_name):
    # Select rows for the sub-sector
    try:
        main = replace_unknown(
            main,
            metric = metric,
            unit = unit,
            sub_sector = sub_sector,
            sub_sub_sector=sub_sub_sector
        )
        mask = main["sub_sub_sector"].str.lower() == subsector_name.lower()
        # rename columns
        sub_data.rename(columns={"Country": "country"}, inplace=True) 
        # For each year column, fill missing values from the sub-sector data
        for col in sub_data.columns:
            if col in main.columns and col not in ["country"]:
                main.loc[mask, col] = main.loc[mask, col].fillna(
                    main.loc[mask, "country"].map(
                        sub_data.set_index("country")[col]
                    )
                )
        return main
    except Exception as e:
        print(f"Error: {e}")

In [ ]:
ele_df = pd.read_csv("/home/deecodes/internship/AfricaEnergy/staging_data/africa_energy_electricity_data.csv")
filt = ele_df.iloc[1025:1079]
filt

ValueError: Cannot index with multidimensional key

In [12]:
fossil_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Fossil-fuels-(MW).xlsx")
fill_subsector_data_unknown(ele_df.iloc[1025:1079], "Installed Capacity in Fossil Fuels", "MW", "Technical", "Electricity, Installed Capacity in Fossil Fuels", fossil_df, "Electricity, Installed Capacity in Fossil Fuels")

/tmp/ipykernel_908991/1234894152.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["metric"] = df["metric"].replace(["Unknown", "unknown", "", "Installed Capacity in Fossil Fuels"], metric)
/tmp/ipykernel_908991/1234894152.py:5: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df["unit"] = df["unit"].replace(["Unknown", "unknown", "", "MW"]).fillna(unit)
/tmp/ipykernel_908991/1234894152.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1025,Algeria,1,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,5846.21,...,17733.73,19574.40,19995.90,21583.90,23923.50,24763.50,25219.50,26019.50,NaN,NaN
1026,Angola,2,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,194.20,...,1376.52,1384.52,2336.86,2336.86,2356.86,2356.86,2356.86,2356.86,NaN,NaN
1027,Benin,3,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,56.00,...,100.00,200.00,200.00,174.00,301.00,301.00,301.00,301.00,NaN,NaN
1028,Botswana,4,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,157.00,...,917.00,917.00,917.00,917.00,917.00,917.00,917.00,917.00,NaN,NaN
1029,Burkina Faso,5,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,48.00,...,296.00,293.00,289.16,289.16,339.16,339.16,389.16,389.16,NaN,NaN
1030,Burundi,6,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,5.50,...,20.50,20.50,50.50,50.50,50.50,50.50,50.50,50.50,NaN,NaN
1031,Cameroon,7,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,71.00,...,592.45,592.45,705.00,705.00,705.00,705.00,706.36,706.36,NaN,NaN
1032,Cape Verde,8,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,76.11,...,145.33,143.34,143.34,140.94,133.50,141.90,141.90,165.00,NaN,NaN
1033,Central African Republic,9,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,21.00,...,22.00,22.00,22.00,22.00,22.00,22.00,72.00,72.00,NaN,NaN
1034,Chad,10,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,29.00,...,282.00,282.00,282.00,282.00,282.00,282.00,282.00,282.00,NaN,NaN


In [17]:
filt = ele_df.iloc[1079:1133]
filt

,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1079,Algeria,1,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1080,Angola,2,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1081,Benin,3,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1082,Botswana,4,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1083,Burkina Faso,5,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1084,Burundi,6,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1085,Cameroon,7,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1086,Cape Verde,8,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1087,Central African Republic,9,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1088,Chad,10,Unknown,NaN,Electricity,Unknown,Unknown,https://africa-energy-portal.org/database,Africa Energy Portal,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
bio_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Bioenergy-(MW).xlsx")

In [32]:
# Define your row range
row_range = list(range(1079, 1133))

# Call your function directly on those rows using .loc
ele_df.loc[row_range] = fill_subsector_data_unknown(
    ele_df.loc[row_range].copy(),
    "Installed Capacity in Bioenergy",
    "MW",
    "Technical",
    "Electricity, Installed Capacity in Bioenergy (MW)",
    bio_df,
    "Electricity, Installed Capacity in Bioenergy (MW)"
)
ele_df.tail(50)


/tmp/ipykernel_848849/1234894152.py:5: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df["unit"] = df["unit"].replace(["Unknown", "unknown", "", "MW"]).fillna(unit)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1083,Burkina Faso,5,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,48.00,...,296.00,293.00,289.16,289.16,339.16,339.16,389.16,389.16,NaN,NaN
1084,Burundi,6,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,5.50,...,20.50,20.50,50.50,50.50,50.50,50.50,50.50,50.50,NaN,NaN
1085,Cameroon,7,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,71.00,...,592.45,592.45,705.00,705.00,705.00,705.00,706.36,706.36,NaN,NaN
1086,Cape Verde,8,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,76.11,...,145.33,143.34,143.34,140.94,133.50,141.90,141.90,165.00,NaN,NaN
1087,Central African Republic,9,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,21.00,...,22.00,22.00,22.00,22.00,22.00,22.00,72.00,72.00,NaN,NaN
1088,Chad,10,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,29.00,...,282.00,282.00,282.00,282.00,282.00,282.00,282.00,282.00,NaN,NaN
1089,Comoros,11,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.70,...,21.60,21.60,21.60,21.60,21.60,21.60,21.60,21.60,NaN,NaN
1090,Congo Democratic Republic,12,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,19.00,...,29.00,29.00,29.00,31.84,31.84,31.84,31.84,75.57,NaN,NaN
1091,Congo Republic,13,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,28.00,...,424.50,424.50,426.66,426.66,426.66,596.66,596.66,596.66,NaN,NaN
1092,Cote d'Ivoire,14,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,625.14,...,1210.14,1325.14,1325.14,1325.14,1355.14,1355.14,1395.14,1576.14,NaN,NaN


In [30]:
ele_df.tail(100)

,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1033,Central African Republic,9,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,21.00,...,22.00,22.00,22.00,22.00,22.00,22.00,72.00,72.00,NaN,NaN
1034,Chad,10,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,29.00,...,282.00,282.00,282.00,282.00,282.00,282.00,282.00,282.00,NaN,NaN
1035,Comoros,11,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,12.70,...,21.60,21.60,21.60,21.60,21.60,21.60,21.60,21.60,NaN,NaN
1036,Congo Democratic Republic,12,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,19.00,...,29.00,29.00,29.00,31.84,31.84,31.84,31.84,75.57,NaN,NaN
1037,Congo Republic,13,Installed Capacity in Fossil Fuels,MW,Electricity,Technical,"Electricity, Installed Capacity in Fossil Fuels",https://africa-energy-portal.org/database,Africa Energy Portal,28.00,...,424.50,424.50,426.66,426.66,426.66,596.66,596.66,596.66,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,20.00,...,172.33,172.33,172.33,172.33,172.33,172.33,219.33,219.33,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,2784.00,...,4649.00,4902.00,5007.00,4774.00,5070.00,5622.00,5622.00,6072.00,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,3.00,...,136.00,136.00,137.08,102.68,102.68,101.08,101.08,101.08,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,86.25,...,172.00,468.60,528.60,523.60,523.60,523.60,524.80,524.80,NaN,NaN


In [18]:
# Option 1: Modify the original dataframe directly by passing indices
def fill_subsector_data_unknown_ele(main, start_idx, end_idx, metric, unit, sub_sector, sub_sub_sector, sub_data, subsector_name):
    try:
        # Replace unknown values in the specific row range
        main.loc[start_idx:end_idx, "metric"] = main.loc[start_idx:end_idx, "metric"].replace(
            ["Unknown", "unknown", "", "Installed Capacity in Fossil Fuels"], metric
        )
        main.loc[start_idx:end_idx, "unit"] = main.loc[start_idx:end_idx, "unit"].replace(
            ["Unknown", "unknown", "", "MW"], unit
        )
        main.loc[start_idx:end_idx, "sub_sector"] = main.loc[start_idx:end_idx, "sub_sector"].replace(
            ["Unknown", "unknown", "", "Technical"], sub_sector
        )
        main.loc[start_idx:end_idx, "sub_sub_sector"] = main.loc[start_idx:end_idx, "sub_sub_sector"].replace(
            ["Unknown", "unknown", "", "Electricity, Installed Capacity in Fossil Fuels"], sub_sub_sector
        )
        
        # Create mask for the specific subsector within the row range
        mask = (main.index >= start_idx) & (main.index <= end_idx) & (main["sub_sub_sector"].str.lower() == subsector_name.lower())
        
        # Rename columns in sub_data
        sub_data = sub_data.rename(columns={"Country": "country"})
        
        # Fill missing values
        for col in sub_data.columns:
            if col in main.columns and col not in ["country"]:
                main.loc[mask, col] = main.loc[mask, col].fillna(
                    main.loc[mask, "country"].map(
                        sub_data.set_index("country")[col]
                    )
                )
        
        return main
    except Exception as e:
        print(f"Error: {e}")
        return main

# Usage:
fossil_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Fossil-fuels-(MW).xlsx")

# For rows 1025 to 1079
ele_df = fill_subsector_data_unknown_ele(
    ele_df, 
    1025, 1079,
    "Installed Capacity in Fossil Fuels", 
    "MW", 
    "Technical", 
    "Electricity, Installed Capacity in Fossil Fuels", 
    fossil_df, 
    "Electricity, Installed Capacity in Fossil Fuels"
)

# For rows 1080 to 1133 (different sub-sector)
renewable_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Bioenergy-(MW).xlsx")
ele_df = fill_subsector_data_unknown_ele(
    ele_df,
    1079, 1133,
    "Installed Capacity in Bioenergy",
    "MW",
    "Technical",
    "Electricity, Installed Capacity in Bioenergy (MW)",
    bio_df,
    "Electricity, Installed Capacity in Bioenergy (MW)"
)

/tmp/ipykernel_908991/365335761.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  main.loc[start_idx:end_idx, "unit"] = main.loc[start_idx:end_idx, "unit"].replace(
/tmp/ipykernel_908991/365335761.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  main.loc[start_idx:end_idx, "unit"] = main.loc[start_idx:end_idx, "unit"].replace(


In [19]:
filt

,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
1079,Algeria,1,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,5846.21,...,17733.73,19574.40,19995.90,21583.90,23923.50,24763.50,25219.50,26019.50,NaN,NaN
1080,Angola,2,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,51.00,51.00,51.00,51.00,51.00,51.00,51.00,51.00,NaN,NaN
1081,Benin,3,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN
1082,Botswana,4,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN
1083,Burkina Faso,5,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,0.90,0.90,0.90,0.90,0.90,0.90,0.90,0.90,NaN,NaN
1084,Burundi,6,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,4.00,...,4.00,4.00,4.00,4.00,4.00,4.00,4.00,4.00,NaN,NaN
1085,Cameroon,7,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN
1086,Cape Verde,8,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN
1087,Central African Republic,9,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,NaN,NaN
1088,Chad,10,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,2.00,...,2.00,2.00,2.00,2.00,2.00,2.00,2.00,2.00,NaN,NaN


In [24]:
nat_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-access-to-electricity-National-(%-of-population).xlsx")
fill_subsector_data_known(ele_df, nat_df, "Population access to electricity-National (% of population)")

Successfully filled data for subsector 'Population access to electricity-National (% of population)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [26]:
urban_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-access-to-electricity-Urban-(%-of-population).xlsx")
fill_subsector_data_known(ele_df, urban_df, "Population access to electricity-Urban (% of population)")

Successfully filled data for subsector 'Population access to electricity-Urban (% of population)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [27]:
natt_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-with-access-to-electricity-National-(millions-of-people).xlsx")
fill_subsector_data_known(ele_df, natt_df, "Population with access to electricity-National (millions of people)")

Successfully filled data for subsector 'Population with access to electricity-National (millions of people)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [28]:
rur_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-without-access-to-electricity-Rural-(millions-of-people) (1).xlsx")
fill_subsector_data_known(ele_df, rur_df, "Population with access to electricity-Rural (millions of people)")

Successfully filled data for subsector 'Population with access to electricity-Rural (millions of people)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [29]:
ur_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-with-access-to-electricity-Urban-(millions-of-people).xlsx")
fill_subsector_data_known(ele_df, ur_df, "Population with access to electricity-Urban (millions of people)")

Successfully filled data for subsector 'Population with access to electricity-Urban (millions of people)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [30]:
wout_nat = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-without-access-to-electricity-National-(millions-of-people).xlsx")
fill_subsector_data_known(ele_df, wout_nat, "Population without access to electricity-National (millions of people)")

Successfully filled data for subsector 'Population without access to electricity-National (millions of people)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [31]:
rural_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-without-access-to-electricity-Rural-(millions-of-people) (1).xlsx")
fill_subsector_data_known(ele_df, rural_df, "Population without access to electricity-Rural (millions of people)")

Successfully filled data for subsector 'Population without access to electricity-Rural (millions of people)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [32]:
urb_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Access_Population-without-access-to-electricity-Urban-(millions-of-people).xlsx")
fill_subsector_data_known(ele_df, urb_df, "Population without access to electricity-Urban (millions of people)")

Successfully filled data for subsector 'Population without access to electricity-Urban (millions of people)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [33]:
exp_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Supply_Electricity-export-(GWh) (1).xlsx")
fill_subsector_data_known(ele_df, exp_df, "Electricity export (GWh)")

Successfully filled data for subsector 'Electricity export (GWh)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [34]:
fin_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Supply_Electricity-final-consumption-(GWh) (1).xlsx")
fill_subsector_data_known(ele_df, fin_df, "Electricity final consumption (GWh)")

Successfully filled data for subsector 'Electricity final consumption (GWh)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [35]:
geo_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Geothermal-(MW).xlsx")
fill_subsector_data_known(ele_df, geo_df, "Electricity installed capacity in Geothermal (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Geothermal (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [36]:
hy_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Hydropower-(MW).xlsx")
fill_subsector_data_known(ele_df, hy_df, "Electricity installed capacity in Hydropower (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Hydropower (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [37]:
non_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Non-renewable-energy-(MW).xlsx")
fill_subsector_data_known(ele_df, non_df, "Electricity installed capacity in Non-renewable energy (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Non-renewable energy (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [38]:
nuc_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Non-renewable-energy-(MW).xlsx")
fill_subsector_data_known(ele_df, nuc_df, "Electricity installed capacity in Nuclear (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Nuclear (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [39]:
sol_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Solar-(MW).xlsx")
fill_subsector_data_known(ele_df, sol_df, "Electricity installed capacity in Solar (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Solar (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [40]:
ren_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Total-renewable-energy-(MW).xlsx")
fill_subsector_data_known(ele_df, ren_df, "Electricity installed capacity in Total renewable energy (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Total renewable energy (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [41]:
wind_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-Wind-(MW).xlsx")
fill_subsector_data_known(ele_df, wind_df, "Electricity installed capacity in Wind (MW)")

Successfully filled data for subsector 'Electricity installed capacity in Wind (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [42]:
rene_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity-in-other-Non-renewable-energy-(MW).xlsx")
fill_subsector_data_known(ele_df, rene_df, "Electricity installed capacity in other Non-renewable energy (MW)")

Successfully filled data for subsector 'Electricity installed capacity in other Non-renewable energy (MW)' (54 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [43]:
total_df = pd.read_excel("/home/deecodes/internship/AfricaEnergy/electricity/Database_Electricity_Technical_Electricity-installed-capacity,-Total-(MW).xlsx")
fill_subsector_data_known(ele_df, total_df, "Electricity installed capacity, Total (MW)")

Successfully filled data for subsector 'Electricity installed capacity, Total (MW)' (53 rows)


,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,NaN,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [53]:
import numpy as np
ele_df["unit"] = ele_df["unit"].replace({pd.NA: "MW", None: "MW", np.nan: "MW"})
ele_df

,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0,NaN,NaN
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5,NaN,NaN
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5,NaN,NaN
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9,NaN,NaN
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1129,Tunisia,51,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1130,Uganda,52,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2,NaN,NaN
1131,Zambia,53,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,NaN,NaN


In [58]:
ele_df.dropna(axis=1, how='all', inplace=True)

In [59]:
ele_df

,country,country_serial,metric,unit,sector,sub_sector,sub_sub_sector,source_link,source,2000,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Algeria,1,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,98.6,...,99.2,99.3,99.4,99.4,99.5,99.6,99.5,99.7,99.8,100.0
1,Angola,2,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,24.2,...,38.4,32.0,42.0,41.8,42.9,45.3,45.6,47.0,48.2,48.5
2,Benin,3,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,21.5,...,34.7,34.1,29.6,37.0,34.5,39.0,40.0,41.0,42.0,56.5
3,Botswana,4,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,26.5,...,58.0,60.0,62.1,64.2,67.4,68.3,70.1,71.8,73.7,75.9
4,Burkina Faso,5,Population access to electricity-National,% of population,Electricity,Access,Population access to electricity-National (% o...,https://africa-energy-portal.org/database,Africa Energy Portal,9.1,...,15.4,19.2,16.2,16.6,17.0,14.4,17.6,18.5,19.0,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1128,Togo,50,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1129,Tunisia,51,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1130,Uganda,52,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,0.0,...,36.0,54.0,64.5,64.5,76.4,96.2,96.2,96.2,96.2,96.2
1131,Zambia,53,Installed Capacity in Bioenergy,MW,Electricity,Technical,"Electricity, Installed Capacity in Bioenergy (MW)",https://africa-energy-portal.org/database,Africa Energy Portal,12.4,...,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8,42.8


In [61]:
ele_df.to_csv("/home/deecodes/internship/AfricaEnergy/staging_data/africa_electricity_data.csv")